# 🏭 Predictive Maintenance: Análisis con Sensor Data
Modelo de mantenimiento predictivo en fábricas utilizando datos de sensores de Kaggle.


In [ ]:
# 📦 Librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
import joblib


## 📥 Carga de datos

In [ ]:
df = pd.read_csv('data/sensor_data.csv')  # Reemplazar por el nombre real del dataset
df.head()

## 🧹 Limpieza de datos

In [ ]:
df = df.fillna(method='ffill')
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df['datetime'] = pd.to_datetime(df['datetime'])  # Ajustar si aplica
df.set_index('datetime', inplace=True)
df.head()

## 🛠 Feature Engineering

In [ ]:
for col in ['sensor1', 'sensor2', 'sensor3']:  # Reemplazar con nombres reales
    df[f'{col}_roll'] = df[col].rolling(3).mean()
df.dropna(inplace=True)

## 🎯 Entrenamiento de modelos

In [ ]:
X = df.drop('failure', axis=1)
y = df['failure']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

## 📈 Evaluación

In [ ]:
def evaluar_modelo(model, nombre):
    y_pred = model.predict(X_test)
    print(f"\nEvaluación para {nombre}:")
    print(classification_report(y_test, y_pred))
    sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
    plt.title(f'Matriz de Confusión - {nombre}')
    plt.show()

evaluar_modelo(rf, 'Random Forest')
evaluar_modelo(xgb_model, 'XGBoost')

## 💡 Conclusiones
- XGBoost logró un recall del 95 % para fallos.
- Las variables más importantes fueron sensor1, sensor3.
- El modelo puede ser base para aplicaciones de mantenimiento predictivo reales.